### Defining Experiment

In [5]:
experiment = {
    
    'experiment_name' : 'AST',
    'collection' : '',
    'taxonomy_type' : '',
    'papers_type' : '_ENHANCED'
}
keywords = (experiment['collection'] == 'keywords')
shortcut = {'_ENHANCED':'enh', '':'std'}

In [6]:
from libs import *
np.set_printoptions(suppress=True)


EXP_DIR = f"experiments/{experiment['experiment_name']}/{experiment['collection']}"
makedir(EXP_DIR)

### Loading Relevance Matrix

In [8]:
# rel_mat = load_obj(
#     f'input_data/relevance_matrices/{experiment["experiment_name"]}\
# /rel_mat_{experiment["experiment_name"]}\
# _{"keywords_" if keywords else ""}\
# {shortcut[experiment["papers_type"]]}_{shortcut[experiment["taxonomy_type"]]}.pkl'
# )

topics_unique = load_obj(name=
                    f'input_data/taxonomies/unique_topics{experiment["taxonomy_type"]}'
                           )

indices_unique = load_obj(name=
                          f"input_data/taxonomies/unique_indices{experiment['taxonomy_type']}")

papers_df = pd.read_csv(f'input_data/text_collections/papers_df{experiment["papers_type"]}.csv',
                            index_col=0)

if keywords:
    papers_df['keywords'] = papers_df['keywords'].apply(eval)
    texts = papers_df['keywords'].apply(lambda x: ' '.join(x)).str.lower().to_numpy()

### Building Similarity Matrix

In [109]:
def diag_mult(v, M):
    result = M.copy()
    for i in range(M.shape[0]):
        result[i] = v[i]*result[i]    
    return result

In [839]:
rel_mat = load_obj(name=f'input_data/relevance_matrices/{experiment["experiment_name"]}/'+
f'rel_mat_{experiment["experiment_name"]}_{"keywords_" if keywords else ""}' + 
f'{shortcut[experiment["papers_type"]]}_{shortcut[experiment["taxonomy_type"]]}.pkl'
)
rel_mat[rel_mat<0.3] = .0
sim_mat = rel_mat.T@rel_mat

In [885]:
import custom_algorithms as custom
reload(custom)

clf1 = custom.FSC(mat_diff=0.0035, derivative_scale=0.005)
clf1.fit(sim_mat)

In [874]:
clf2 = custom.LaplacianFCM(n_clusters=10)
clf2.fit(sim_mat)

In [864]:
mm = clf.mm
z = clf.mm.sum(axis=1)
z[z==0.] = 1.
mm /= z[:, None]

i = 9
cl = mm[:, i]
idx = np.argsort(-cl)

for el, u in zip(topics_unique[idx], cl[idx]):
    if u > 0:
        print(f"{el} :: {u}")

Kernel Independent components :: 1.0
Contingency table analysis :: 1.0
Canonical correlation analysis :: 1.0
Time series analysis :: 0.5528505502974652
Independent Component Analysis :: 0.5475337521576339
Survival analysis :: 0.5410369663735914
Traffic analysis :: 0.5083412219667854
Content analysis and feature selection :: 0.4423547430767658
Sentiment analysis :: 0.43379705212812203
Factor analysis :: 0.4075110735211057
Web log analysis :: 0.39102535761347407
Query log analysis :: 0.36140456708896646
Cluster analysis :: 0.32743534967052573
Hierarchical representations :: 0.23847201769687726
Robust regression :: 0.07585180166603327


In [882]:
mm = clf2.mm
z = clf.mm.sum(axis=1)
z[z==0.] = 1.
mm /= z[:, None]

i = 7
cl = mm[:, i]
idx = np.argsort(-cl)

for el, u in zip(topics_unique[idx], cl[idx]):
    if u > 0:
        print(f"{el} :: {u}")

Geographic visualization :: 0.9997529503669619
Scientific visualization :: 0.995532107244869
Empirical studies in visualization :: 0.9953065543690983
Visualization theory concepts and paradigms :: 0.9896888018750237
Visualization toolkits :: 0.9870548505974973
Treemaps :: 0.9756558649044922
Data encoding and canonicalization :: 0.9744046713128727
Visual analytics :: 0.5111748656287197
Non-negative matrix factorization :: 0.2538943099384111
Personalization :: 0.17755880353601397
Visualization design and evaluation methods :: 0.13879986289841054
Factor graphs :: 0.0904590259684924
Summarization :: 0.08741182084153645
Sample complexity and generalization bounds :: 0.047149007266958434
Record and block layout :: 0.04516260839306284
Graph based conceptual clustering :: 0.0356872777468378
Partially-observable Markov decision processes :: 0.033954263316484073
Query optimization :: 0.028064494519685592
Multi-agent reinforcement learning :: 0.0271352346678958
Conceptual clustering :: 0.02671746

### Clustering 

In [5]:
from sklearn.manifold import spectral_embedding
from sklearn.cluster import KMeans
from fcmeans import FCM
from LaplacianFCM import LaplacianFCM
from sklearn.decomposition import PCA
import utils

clf = LaplacianFCM(n_clusters=12, embedding_dim=12, random_state=1)

clf.fit(sim_mat)

In [6]:
import utils 
reload(utils)
from utils import Clusters

cl_fcm = Clusters(clf.fcm.u.copy(), topics_unique, clf.embedding, random_state=1)
cl_km = Clusters(clf.kmeans.u.copy(), topics_unique, clf.embedding, random_state=1)

### Lifting